In [1]:
import hsfs
import hopsworks
import pandas as pd

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings("ignore")
#RvLmP6cByDVIO2do.eUiPUAajvVgkNKBoBKswguvGuCb3UvWY9lnuw832q5mYKbC5eD4en9QbdsntpyDq

In [2]:
project = hopsworks.login()
fs = project.get_feature_store() 

air_columns_names = ['aqi', 'iaqi_h', 'iaqi_p', 'iaqi_pm10', 'iaqi_t', 'date',
                'o3_avg', 'o3_max', 'o3_min', 'pm10_avg', 'pm10_max', 'pm10_min',
                'pm25_avg', 'pm25_max', 'pm25_min']


air_quality_fg = fs.get_or_create_feature_group(
    name = 'air_quality_fg',
    version = 1
)
weather_fg = fs.get_or_create_feature_group(
    name = 'weather_fg',
    version = 1
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/5359
Connected. Call `.close()` to terminate connection gracefully.


In [3]:
query = air_quality_fg.select(air_columns_names).join(weather_fg.select_all() , on=['date'])

# air_columns = air_quality_fg.select_all().show(5).columns
# weather_columns = weather_fg.select_all().show(5).columns

In [4]:
query_show = query.show(5)
col_names = query_show.columns

2023-01-13 10:06:36,021 INFO: USE `id2223iris_featurestore`
2023-01-13 10:06:36,638 INFO: WITH right_fg0 AS (SELECT *
FROM (SELECT `fg1`.`aqi` `aqi`, `fg1`.`iaqi_h` `iaqi_h`, `fg1`.`iaqi_p` `iaqi_p`, `fg1`.`iaqi_pm10` `iaqi_pm10`, `fg1`.`iaqi_t` `iaqi_t`, `fg1`.`date` `date`, `fg1`.`o3_avg` `o3_avg`, `fg1`.`o3_max` `o3_max`, `fg1`.`o3_min` `o3_min`, `fg1`.`pm10_avg` `pm10_avg`, `fg1`.`pm10_max` `pm10_max`, `fg1`.`pm10_min` `pm10_min`, `fg1`.`pm25_avg` `pm25_avg`, `fg1`.`pm25_max` `pm25_max`, `fg1`.`pm25_min` `pm25_min`, `fg1`.`city` `join_pk_city`, `fg1`.`date` `join_pk_date`, `fg1`.`date` `join_evt_date`, `fg0`.`city` `city`, `fg0`.`tempmax` `tempmax`, `fg0`.`tempmin` `tempmin`, `fg0`.`temp` `temp`, `fg0`.`feelslikemax` `feelslikemax`, `fg0`.`feelslikemin` `feelslikemin`, `fg0`.`feelslike` `feelslike`, `fg0`.`dew` `dew`, `fg0`.`humidity` `humidity`, `fg0`.`precip` `precip`, `fg0`.`precipprob` `precipprob`, `fg0`.`precipcover` `precipcover`, `fg0`.`snow` `snow`, `fg0`.`snowdepth` `snow

In [5]:
query_show

,aqi,iaqi_h,iaqi_p,iaqi_pm10,iaqi_t,date,o3_avg,o3_max,o3_min,pm10_avg,...,windgust,windspeed,winddir,pressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,conditions
0,34,79.0,1026.4,14,12.7,1672552800000,11,21,17,27,...,18.7,8.9,25.5,1023.7,55.1,10.1,98.5,8.5,6,Partially cloudy
1,53,52.0,1028.8,21,17.7,1672639200000,13,25,6,19,...,11.9,14.3,198.1,1024.2,81.0,10.0,67.7,5.9,3,"Rain, Partially cloudy"
2,46,59.7,1025.5,46,13.3,1672725600000,11,13,11,15,...,20.2,7.7,8.2,1030.0,35.1,10.0,106.3,9.2,5,Partially cloudy
3,65,69.0,1027.4,25,9.4,1672812000000,5,19,3,33,...,10.4,7.4,350.9,1033.5,2.4,10.0,92.3,7.9,5,Clear
4,78,77.0,1027.6,30,10.0,1672898400000,8,24,2,25,...,11.5,5.7,12.9,1029.7,0.1,11.2,111.3,9.6,5,Clear


In [6]:
# col_names = list(air_columns) + list(weather_columns)

category_cols = ['city','date','conditions','aqi']
mapping_transformers = {col_name:fs.get_transformation_function(name='standard_scaler') for col_name in col_names if col_name not in category_cols}
category_cols = {col_name:fs.get_transformation_function(name='label_encoder') for col_name in category_cols if col_name not in ['date','aqi']}
mapping_transformers.update(category_cols)

In [7]:

feature_view = fs.create_feature_view(
    name = 'air_quality_fv',
    version = 5,
    transformation_functions = mapping_transformers,
    query = query
)

feature_view = fs.get_feature_view(
    name = 'air_quality_fv',
    version = 5
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/5359/fs/5272/fv/air_quality_fv/version/5
